# Install ISR

Adapted by Ozgun Haznedaar from ISR_Training_Tutorial.ipynb: https://github.com/
idealo/image-super-resolution/tree/master/notebooks




The purpose of this notebook is to set up a model from the ISR repo and train it.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture
%cd /content/drive/MyDrive/image-super-resolution
!python setup.py install
!pip install 'h5py==2.10.0' --force-reinstall
!pip install -U PyYAML
!pip install imagecodecs


# if using local repo of ISR
# else use !pip install ISR and see ISR_module_adjustments notebook for changes
# to run locally
#!pip install gast>=0.3.2
#!pip install ISR

# Train

In [ ]:
from ISR.models import RDN
from ISR.models import Discriminator
from ISR.models import Cut_VGG19

In [ ]:
lr_train_patch_size = 60
layers_to_extract = [5, 9]
scale = 3
hr_train_patch_size = lr_train_patch_size * scale

rdn  = RDN(arch_params={'C':3, 'D':10, 'G':64, 'G0':64, 'x': scale }, patch_size=lr_train_patch_size)

#rdn  = RDN(arch_params={'C':6, 'D':20, 'G':64, 'G0':64, 'x': scale }, patch_size=lr_train_patch_size) # for large RDN

f_ext = Cut_VGG19(patch_size=hr_train_patch_size, layers_to_extract=layers_to_extract)
discr = Discriminator(patch_size=hr_train_patch_size, kernel_size=3)

## Give the models to the Trainer
The Trainer object will combine the networks, manage your training data and keep you up-to-date with the training progress through Tensorboard and the command line.

Here we do not use  the pixel-wise MSE but only the perceptual loss by specifying the respective weights in `loss_weights`

In [ ]:
from ISR.train import Trainer
loss_weights = {
  'generator': 1.0, #0.0
  'feature_extractor': 0.0, #0.0833
  'discriminator': 0.00 #0.01
}
losses = {
  'generator': 'mae',
  'feature_extractor': 'mse',
  'discriminator': 'binary_crossentropy'
} 

log_dirs = {'logs': '/content/drive/MyDrive/logs', 'weights': '/content/drive/MyDrive/weights'}

learning_rate = {'initial_value': 0.0004, 'decay_factor': 0.5, 'decay_frequency': 30}

flatness = {'min': 0.0, 'max': 0.15, 'increase': 0.01, 'increase_frequency': 5}

trainer = Trainer(
    generator=rdn,
    discriminator=discr,
    feature_extractor=f_ext,
    lr_train_dir='PATH TO LR TRAINING DATA',
    hr_train_dir='PATH TO HR TRAINING DATA',
    lr_valid_dir='PATH TO LR VALIDATION DATA',
    hr_valid_dir='PATH TO HR VALIDATION DATA',
    loss_weights=loss_weights,
    learning_rate=learning_rate,
    flatness=flatness,
    dataname='div2kx3_rdn_small',
    log_dirs=log_dirs,
    weights_generator=None,
    weights_discriminator=None,
    n_validation=60,
)


Choose epoch number, steps and batch size and start training

In [ ]:
trainer.train(
    epochs=150,
    steps_per_epoch=20,
    batch_size=4,
    monitored_metrics={'val_generator_PSNR_Y':'max','val_discriminator_loss':'min', 'val_feature_extractor_loss':'min'}
)